In [375]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("azminetoushikwasi/supplygraph-supply-chain-planning-using-gnns")

# print("Path to dataset files:", path)

In [376]:
import pandas as pd
path = "./supplygraph-supply-chain-planning-using-gnns/versions/2/Raw Dataset/"

# Nodes
nodes = pd.read_csv(path + "Nodes/Nodes.csv")   
edges = pd.read_csv(path + "Edges/Edges (Plant).csv")

delivery_to_distributor = pd.read_csv(path + "Temporal Data/Unit/Delivery To distributor.csv")
factory_issue = pd.read_csv(path + "Temporal Data/Unit/factory issue.csv")
production = pd.read_csv(path + "Temporal Data/Unit/Production .csv")
sales_order = pd.read_csv(path + "Temporal Data/Unit/Sales order.csv")



In [377]:
import pandas as pd
import numpy as np
from torch_geometric_temporal.signal import StaticGraphTemporalSignal

path = "./supplygraph-supply-chain-planning-using-gnns/versions/2/Raw Dataset/Temporal Data/Unit/"

production = pd.read_csv(path + "Production .csv")
factory_issue = pd.read_csv(path + "factory issue.csv")
delivery = pd.read_csv(path + "Delivery To distributor.csv")
sales_order = pd.read_csv(path + "Sales order.csv")

products = [col for col in production.columns if col != "Date"]
product_to_id = {prod: i for i, prod in enumerate(products)}

def transform_temporal(df, mapping):
    df_no_date = df.drop(columns=["Date"])
    return df_no_date.rename(columns=mapping)

X_prod = transform_temporal(production, product_to_id)
X_issue = transform_temporal(factory_issue, product_to_id)
X_delivery = transform_temporal(delivery, product_to_id)
X_sales = transform_temporal(sales_order, product_to_id)  

X_prod_np = X_prod.to_numpy()
X_issue_np = X_issue.to_numpy()
X_delivery_np = X_delivery.to_numpy()
X_sales_np = X_sales.to_numpy()

# shape: [T, N, F] = [time_steps, num_nodes, num_features]
X = np.stack([X_prod_np, X_issue_np, X_delivery_np, X_sales_np], axis=-1)

y = X_prod_np[1:]       
X = X[:-1]

print("X shape:", X.shape)  # (T-1, N, 4)
print("y shape:", y.shape)  # (T-1, N)


X shape: (220, 41, 4)
y shape: (220, 41)


In [378]:
from sklearn.preprocessing import StandardScaler

# Flatten for scaling: combine time and nodes
T, N, F = X.shape
X_flat = X.reshape(-1, F)  # shape [T*N, F]
y_flat = y.reshape(-1, 1)  # shape [T*N, 1]

# Create scalers
scaler_X = StandardScaler()
scaler_y = StandardScaler()

# Fit and transform
X_scaled_flat = scaler_X.fit_transform(X_flat)
y_scaled_flat = scaler_y.fit_transform(y_flat)

# Reshape back to original shape
X_scaled = X_scaled_flat.reshape(T, N, F)
y_scaled = y_scaled_flat.reshape(T, N)

print(f"X_scaled range: {X_scaled.min():.2f} to {X_scaled.max():.2f}")
print(f"y_scaled range: {y_scaled.min():.2f} to {y_scaled.max():.2f}")


X_scaled range: -0.43 to 11.69
y_scaled range: -0.38 to 8.12


In [379]:
edges = pd.read_csv("./supplygraph-supply-chain-planning-using-gnns/versions/2/Raw Dataset/Edges/Edges (Plant).csv")

edges['node1'] = edges['node1'].astype(str)
edges['node2'] = edges['node2'].astype(str)

edges['node1'] = edges['node1'].map(product_to_id)
edges['node2'] = edges['node2'].map(product_to_id)

edge_index = edges[['node1', 'node2']].to_numpy().T.astype(np.int64)

num_nodes = len(product_to_id)

dataset = StaticGraphTemporalSignal(
    edge_index=edge_index,
    edge_weight=np.ones(edge_index.shape[1]),
    features=X_scaled,
    targets=y_scaled
)


In [380]:
y.shape

(220, 41)

In [381]:
edges

,Plant,node1,node2
0,1901,29,25
1,1903,23,25
2,1903,23,27
3,1903,23,32
4,1903,23,30
...,...,...,...
1642,2122,21,19
1643,2122,21,12
1644,2122,0,19
1645,2122,0,12


In [382]:
from torch_geometric_temporal.signal import temporal_signal_split

train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.7)

In [383]:
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import GCLSTM

class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        self.recurrent = GCLSTM(node_features, 64, 1)
        self.linear = torch.nn.Linear(64, 1)

    def forward(self, x, edge_index, edge_weight):
        h, c = self.recurrent(x, edge_index, edge_weight)  # Unpack (H, C)
        h = F.relu(h)
        h = self.linear(h)
        return h.squeeze(-1)

In [384]:
model = RecurrentGCN(node_features=4)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [385]:
from tqdm import tqdm

def calculate_mse(model, dataset):
    """Calculate MSE on a dataset"""
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for time, snapshot in enumerate(dataset):
            y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
            total_loss += torch.mean((y_hat - snapshot.y) ** 2).item()
    model.train()
    return total_loss / (time + 1)

n_epoch = 100  # Number of epochs
for epoch in tqdm(range(n_epoch)):
    model.train()
    cost = 0
    for time, snapshot in enumerate(train_dataset):
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)  # Model prediction
        cost += torch.mean((y_hat - snapshot.y) ** 2)                       # MSE loss
    cost = cost / (time + 1)                                                # Average cost
    cost.backward()                                                         # Backpropagation
    optimizer.step()                                                        # Update weights
    optimizer.zero_grad()                                                   # Reset gradients
    
    # Print metrics every 10 epochs
    if (epoch + 1) % 10 == 0:
        train_mse = calculate_mse(model, train_dataset)
        test_mse = calculate_mse(model, test_dataset)
        print(f"\nEpoch {epoch + 1}/{n_epoch}")
        print(f"Train MSE: {train_mse:.4f}")
        print(f"Test MSE: {test_mse:.4f}")

 10%|█         | 10/100 [00:07<01:21,  1.10it/s]


Epoch 10/100
Train MSE: 0.3698
Test MSE: 0.2855


 20%|██        | 20/100 [00:14<01:09,  1.16it/s]


Epoch 20/100
Train MSE: 0.3103
Test MSE: 0.3118


 30%|███       | 30/100 [00:25<01:12,  1.04s/it]

...


Epoch 80/100
Train MSE: 0.2330
Test MSE: 0.2415


 90%|█████████ | 90/100 [01:08<00:07,  1.26it/s]


Epoch 90/100
Train MSE: 0.2308
Test MSE: 0.2410


100%|██████████| 100/100 [01:15<00:00,  1.32it/s]


Epoch 100/100
Train MSE: 0.2286
Test MSE: 0.2410
